# Домашнее задание к лекции "Статистика. Практика"

In [1]:
import pandas as pd
from scipy import stats
import glob
import re

## Задание 1

In [2]:
df = pd.read_csv('vgsales.csv')
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['Genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', nan], dtype=object)

### Критики положительно относятся к спортивным играм и выставляют им оценки выше 80?

H0: средняя оценка критиков спортивным играм <= 80  
H1: средняя оценка критиков спортивным играм > 80

In [4]:
alpha = 0.05
df_sport_critic = df[df['Genre'] == 'Sports']['Critic_Score'].dropna()
result = stats.ttest_1samp(df_sport_critic, 80, alternative='greater')

print(df_sport_critic.mean())
print(result)

if result.pvalue < alpha: 
    print('Отвергаем нулевую гипотезу, среднее больше 80 (критики положительно относятся к спортивным играм)')
else:
    print('Не отвергаем нулевую нулевую гипотезу, критики отрицательно относятся к спортивным играм')

71.96817420435511
Ttest_1sampResult(statistic=-19.99751475320892, pvalue=1.0)
Не отвергаем нулевую нулевую гипотезу, критики отрицательно относятся к спортивным играм


### Критикам нравятся больше игры на PC или на PS4?

In [5]:
alpha = 0.05
df_PC_critic = df[df['Platform'] == 'PC']['Critic_Score'].dropna()
df_PS4_critic = df[df['Platform'] == 'PS4']['Critic_Score'].dropna()
result = stats.ttest_ind(df_PC_critic, df_PS4_critic, equal_var=False)

print(df_PC_critic.mean())
print(df_PS4_critic.mean())
print(result)
higher_mean = ''

if df_PC_critic.mean() > df_PS4_critic.mean():
    higher_mean = 'PC'
else:
    higher_mean = 'PS4'

if result.pvalue < alpha: 
    print(f'Отвергаем нулевую гипотезу, критики по разному относятся к играм на PC и на PS4. Судя по средним значениям критики лучше относятся к играм на {higher_mean}.')
else:
    print('Не отвергаем нулевую нулевую гипотезу')

75.92867132867133
72.09126984126983
Ttest_indResult(statistic=4.3087588262138725, pvalue=2.067249157283479e-05)
Отвергаем нулевую гипотезу, критики по разному относятся к играм на PC и на PS4. Судя по средним значениям критики лучше относятся к играм на PC.


### Критикам больше нравятся стрелялки или стратегии?

In [6]:
alpha = 0.05
df_Shooter_critic = df[df['Genre'] == 'Shooter']['Critic_Score'].dropna()
df_Strategy_critic = df[df['Genre'] == 'Strategy']['Critic_Score'].dropna()
result = stats.ttest_ind(df_PC_critic, df_PS4_critic, equal_var=False)

print(df_Shooter_critic.mean())
print(df_Strategy_critic.mean())
print(result)

if df_Shooter_critic.mean() > df_Strategy_critic.mean():
    higher_mean = 'Shooter'
else:
    higher_mean = 'Strategy'

if result.pvalue < alpha: 
    print(f'Отвергаем нулевую гипотезу, критики по разному относятся к стрелялкам и стратегиям. Судя по средним значениям критики лучше относятся к играм жанра {higher_mean}.')
else:
    print('Не отвергаем нулевую нулевую гипотезу')

70.18114406779661
72.08609271523179
Ttest_indResult(statistic=4.3087588262138725, pvalue=2.067249157283479e-05)
Отвергаем нулевую гипотезу, критики по разному относятся к стрелялкам и стратегиям. Судя по средним значениям критики лучше относятся к играм жанра Strategy.


## Задание 2

In [70]:
spam_initial = pd.read_csv('spam.csv')
spam = pd.read_csv('spam.csv')
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
# приведем все слова к нижнему регистру
spam['Message'] = spam['Message'].str.lower()
spam.head()

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [9]:
# уберем все служебные символы
spam['Message'].replace(to_replace='[\W_]+', value=' ', inplace=True, regex=True)
spam.head()

,Category,Message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives arou...


In [10]:
# уберем все стоп-слова
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))

message_splitted = [line.split() for line in spam['Message'].tolist()]

without_sw = [[word for word in line if word not in stopwords_set] for line in message_splitted]

In [11]:
# привидем все слова к нормальной форме
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
lemmatized_spam = [[wordnet_lemmatizer.lemmatize(word) for word in text] for text in without_sw]

In [12]:
# вернем сообщения в датафрейм
spam['Message'] = [' '.join(line) for line in lemmatized_spam]

In [86]:
# переведем категории в числа 0 и 1
def categories_to_nums(cell):
    if cell == 'ham':
        return 0
    elif cell == 'spam':
        return 1

target = spam['Category'].apply(categories_to_nums)

In [87]:
# преобразуем все сообщения в вектора TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(spam['Message'])
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
# разделяем выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, target, test_size=0.25, random_state=42)

In [89]:
# импортируем LDA-функционал
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=42)

In [90]:
# обучаем данные
lr.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [91]:
# делаем прогноз на тестовой выборке
y_pred = lr.predict(X_test)

In [92]:
# расчет точности - отношение верный прогнозов к общему количеству позиций
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9633883704235463

In [93]:
# построим confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1207,    0],
       [  51,  135]])

In [144]:
# построим датафрейм с фактическими и прогнозными значениями
result = pd.DataFrame([y_test]).T
result['Predicted'] = y_pred
result[result['Category'] != result['Predicted']].shape

(51, 2)

In [145]:
# переведем категории обратно в строки
def nums_to_categories(cell):
    if cell == 0:
        return 'ham'
    elif cell == 1:
        return 'spam'

result['Category'] = result['Category'].apply(nums_to_categories)
result['Predicted'] = result['Predicted'].apply(nums_to_categories)
result.head()

,Category,Predicted
3245,ham,ham
944,ham,ham
1044,ham,ham
2484,ham,ham
812,ham,ham


In [146]:
# отфильтруем только неправильно спрогнозированные строки
result = result[result['Category'] != result['Predicted']]

In [147]:
# добавим колонку с исходным текстом сообщений
result = result.join(spam_initial['Message'])
result

,Category,Predicted,Message
881,spam,ham,Reminder: You have not downloaded the content ...
1961,spam,ham,Guess what! Somebody you know secretly fancies...
3864,spam,ham,Oh my god! I've found your number again! I'm s...
2575,spam,ham,Your next amazing xxx PICSFREE1 video will be ...
3548,spam,ham,Rock yr chik. Get 100's of filthy films &XXX p...
2402,spam,ham,Babe: U want me dont u baby! Im nasty and have...
4527,spam,ham,"I want some cock! My hubby's away, I need a re..."
15,spam,ham,"XXXMobileMovieClub: To use your credit, click ..."
2663,spam,ham,Hello darling how are you today? I would love ...
1022,spam,ham,Guess what! Somebody you know secretly fancies...
